<a href="https://colab.research.google.com/github/VictorCampelo/Circle-Points-For-Coloring/blob/main/circulo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Bibliotecas

In [ ]:
!apt install libcairo2-dev
!pip install pycairo
!apt-get install build-essential

In [ ]:
!pip install --upgrade Pillow

In [ ]:
!pip install opencv-python-headless

In [ ]:
!git clone https://github.com/fjean/pymeanshift.git

In [ ]:
cd pymeanshift

In [ ]:
!chmod a+x setup.py
!./setup.py install

## Segmentação

In [ ]:
cd /content/drive/MyDrive/Colab/ciclism/MSS

In [ ]:
!g++ Maincpp.cpp `pkg-config --cflags --libs opencv`
!./a.out /content/drive/MyDrive/Colab/ciclism/images/input/pp.jpg /content/drive/MyDrive/Colab/ciclism/images/intermediate/segmented/pp.png

## Importações

In [ ]:
import cv2
from matplotlib import pyplot as plt 
import numpy as np

from random import random
import cairo
import math
%matplotlib inline
import time

In [ ]:
nome = 'pp'

In [ ]:
import cv2
 
img = cv2.imread(f"/content/drive/MyDrive/Colab/ciclism/images/input/{nome}.jpg", cv2.IMREAD_UNCHANGED)
 
print('Original Dimensions : ',img.shape)
 
scale_percent = 60 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
  
# resize image
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
 
print('Resized Dimensions : ',resized.shape)

cv2.imwrite(f"/content/drive/MyDrive/Colab/ciclism/images/input/{nome}.png", resized)

In [ ]:
import cv2
import pymeanshift as pms

original_image = cv2.imread(f"/content/drive/MyDrive/Colab/ciclism/images/input/{nome}.png")

(segmented_image, labels_image, number_regions) = pms.segment(original_image, spatial_radius=9, 
                                                              range_radius=3, min_density=20)


In [ ]:
cv2.imwrite(f"/content/drive/MyDrive/Colab/ciclism/images/intermediate/segmented/{nome}.png", segmented_image)

In [ ]:
import cv2
 
img = cv2.imread(f"/content/drive/MyDrive/Colab/ciclism/images/intermediate/segmented/{nome}.png", cv2.IMREAD_UNCHANGED)
 
print('Original Dimensions : ',img.shape)
 
scale_percent = 500 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
  
# resize image
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
 
print('Resized Dimensions : ',resized.shape)

segmented_image = resized

cv2.imwrite(f"/content/drive/MyDrive/Colab/ciclism/images/intermediate/segmented/{nome}.png", segmented_image)

In [ ]:
plt.rcParams["figure.figsize"] = (25,15)
plt.imshow(cv2.cvtColor(segmented_image,cv2.COLOR_BGR2RGB))
plt.show()

# Backgroud

In [ ]:
img = cv2.imread('/content/drive/MyDrive/Colab/ciclism/images/intermediate/segmented/'+nome+'.png')

In [ ]:
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
blurred = cv2.GaussianBlur(img, (5, 5), 0) # Remove noise
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
def edgedetect (channel):
    sobelX = cv2.Sobel(channel, cv2.CV_16S, 1, 0)
    sobelY = cv2.Sobel(channel, cv2.CV_16S, 0, 1)
    sobel = np.hypot(sobelX, sobelY)

    sobel[sobel > 255] = 255;
    return sobel

In [ ]:
edgeImg = np.max( [ edgedetect(blurred[:,:, 0]), edgedetect(blurred[:,:, 1]), edgedetect(blurred[:,:, 2]) ], axis=0 )

In [ ]:
plt.imshow(edgeImg,cmap='gray')
plt.show()

In [ ]:
mean = np.mean(edgeImg);
# Zero any value that is less than mean. This reduces a lot of noise.
edgeImg[edgeImg <= mean] = 0;
plt.imshow(edgeImg,cmap='gray')
plt.show()

In [ ]:
def findSignificantContours (img, edgeImg):
    contours, heirarchy = cv2.findContours(edgeImg, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Find level 1 contours
    level1 = []
    for i, tupl in enumerate(heirarchy[0]):
        # Each array is in format (Next, Prev, First child, Parent)
        # Filter the ones without parent
        if tupl[3] == -1:
            tupl = np.insert(tupl, 0, [i])
            level1.append(tupl)
    significant = []
    tooSmall = edgeImg.size * 5 / 100 # If contour isn't covering 5% of total area of image then it probably is too small
    for tupl in level1:
        contour = contours[tupl[0]];
        area = cv2.contourArea(contour)
        if area > tooSmall:
            significant.append([contour, area])

    significant.sort(key=lambda x: x[1])
    #print ([x[1] for x in significant]);
    return [x[0] for x in significant];

In [ ]:
# edgeImg_8u = np.asarray(edgeImg, np.uint8)

# # Find contours
# significant = findSignificantContours(img, edgeImg_8u)

In [ ]:
# # Mask
# mask = edgeImg.copy()
# mask[mask > 0] = 0
# cv2.fillPoly(mask, significant, 255)
# # Invert mask
# mask = np.logical_not(mask)

# #Finally remove the background
# img[mask] = 200;
# img[np.logical_not(mask)] = 0;

In [ ]:
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
cv2.imwrite('/content/drive/MyDrive/Colab/ciclism/images/intermediate/background/'+nome+'.png',img)

# Circlimos

In [ ]:
def load_image(path):
    Image1 = cv2.imread(path)
    Image1 = cv2.cvtColor(Image1, cv2.COLOR_BGR2RGB)
    return Image1
def show_image(image, gray=False):
    if gray:
        plt.imshow(image, cmap='gray')
    else:
        plt.imshow(image)
    plt.show()

In [ ]:
input_image = load_image('/content/drive/MyDrive/Colab/ciclism/images/intermediate/segmented/'+nome+'.png')
show_image(input_image)

In [ ]:
def process_image(image):
    canny = cv2.Canny(image,200,300)
    show_image(canny, gray=True)
    
    edges_inv = (255 - canny)
    show_image(edges_inv,gray=True)
    
    dist_transform = cv2.distanceTransform(edges_inv,cv2.DIST_L2,0) 
    show_image(dist_transform,gray=True)
    
    return dist_transform

In [ ]:
SIZE_X = input_image.shape[1]
SIZE_Y = input_image.shape[0]
SIZE = min(SIZE_X,SIZE_Y)

LINEWIDTH = 0.1

TWOPI = 2.0*3.14
BACK = [255,0,0,0]

In [ ]:
def add_new_circles(is_fill, dist_map, circles,r,t,e):
    for x in range(2*r,SIZE_X- r):
        for y in range(2*r,SIZE_Y - r):
            a=True
            if dist_map[y,x]>r:
                p=(int)((dist_map[y,x]+1)/2)
                if p>e:
                    p=r
                if (is_fill[x,y]==0)&(x-p>=0)&(x+p<SIZE_X)&(y-p>=0)&(y+p<SIZE_Y):
                    for rt in range(x-p,x+p+1):
                        if rt>SIZE_X:
                            break
                        yu=rt-x
                        for ty in range(y-p,y+p+1):
                            if ty>SIZE_Y:
                                break
                            op=ty-y
                            if yu*yu+op*op<(p+1)*(p+1):
                                if is_fill[rt,ty]==1:
                                    a=False
                                    break
                            if a==False:
                                break
                    
                    if a==True:
                        circles.append({
                        'x': x, 
                        'y': y, 
                        'r': p,
                        }) 
                        for st in range(x-p,x+p+1):
                            ui=st-x
                            if st>SIZE_X:
                                break
                            for en in range(y-p,y+p+1):
                                iu=en-y
                                if en>SIZE_Y:
                                    break
                                if ui*ui+iu*iu<=(p+1)*(p+1):
                                    is_fill[st,en]=1    

                        
                        y=y+p*2+r;

def show(img_clr, back, ctx, circles):
    for c in circles:
        if (not np.array_equal(back[int(c['y']),int(c['x'])],np.array([200,200,200]))):
            rgb = img_clr[int(c['y']),int(c['x'])]
            rgba = [rgb[0]/255, rgb[1]/255,rgb[2]/255, 1.0]
            ctx.set_source_rgba(*rgba)
            ctx.arc(c['x'], c['y'], c['r'], 0, TWOPI)
            ctx.fill()
            ctx.set_source_rgba(*[0,0,0,1])
            ctx.arc(c['x'], c['y'], c['r'], 0, TWOPI)
            ctx.stroke()
        else:
            rgb = img_clr[int(c['y']),int(c['x'])]
            rgba = [rgb[0]/255, rgb[1]/255,rgb[2]/255, 0.7]
            ctx.set_source_rgba(*rgba)
            ctx.arc(c['x'], c['y'], c['r'], 0, TWOPI)
            ctx.fill()
            ctx.set_source_rgba(*[0,0,0,1])

In [ ]:
def run_circlism(image):
    processed_image = process_image(image)
    back = cv2.imread('/content/drive/MyDrive/Colab/ciclism/images/intermediate/background/'+nome+'.png')
    s = time.time()
    image = cairo.ImageSurface.create_from_png('/content/drive/MyDrive/Colab/ciclism/images/intermediate/background/'+nome+'.png')
    buffer_surf = cairo.ImageSurface(cairo.FORMAT_ARGB32, SIZE_X, SIZE_Y)
    buffer = cairo.Context(buffer_surf)
    # buffer.set_source_surface(image, 0,0)
    # buffer.set_source_rgba(0,0,0,1)
    buffer.paint()

    # buffer.scale(SIZE_X, SIZE_Y)

    buffer.rectangle(0.0, 0.0, 1.0, 1.0)
    buffer.fill()
    circles = []
    is_fill = np.zeros([SIZE_X+1,SIZE_Y+1])
    D = [ 300, 200, 100, 50, 40, 30, 20, 15, 10, 5]
    # D = [ 40,30 ]
    # add_new_circles(is_fill, processed_image, circles,40,40,100)
    print(time.time()-s)
    for i in range(len(D)):
        if i==0:
            continue
        add_new_circles(is_fill, processed_image, circles,D[i],D[i],D[i-1])
        print(time.time()-s)

    buffer.set_line_width(1)
    show(input_image, back, buffer, circles)
    print (len(circles))
    buffer_surf.write_to_png('/content/drive/MyDrive/Colab/ciclism/images/output/'+nome+'.png')
    print (time.time()-s)

In [ ]:
run_circlism(input_image)

In [ ]:
result = cv2.imread('/content/drive/MyDrive/Colab/ciclism/images/output/'+nome+'.png')
result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(12,12))
plt.imshow(result)
plt.title('EDT Circle Packing')
plt.show()

# Vectorize

# Number

In [ ]:
cd /content/drive/MyDrive/Colab/ciclism/number

In [ ]:
!python run.py /content/drive/MyDrive/Colab/ciclism/images/output/pp.png 40